<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Recursive_Ontology_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Recursive Ontology Compiler — end-to-end single script

Features:
- Iterative compiler with semantic and metric Ω gates
- Alpha ladder (multiple Ω phases) with per-phase snapshots and rollback
- Cycle detection, guard limits, dead-end detection, overflow handling
- Observability via on_step callback
- Tracker-ready rows and CSV export
- Smoke tests to validate convergence, cycles, overflow, and rollback

Run:
  python ontology_compiler.py --csv out.csv --verbose

"""

from __future__ import annotations

import argparse
import csv
import math
import sys
from copy import deepcopy
from dataclasses import dataclass, field
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Set


# ------------------------------
# Math helpers for metric gates
# ------------------------------

def dot(a: Sequence[float], b: Sequence[float]) -> float:
    return sum(x * y for x, y in zip(a, b))


def norm(a: Sequence[float]) -> float:
    return math.sqrt(dot(a, a))


def cosine_similarity(a: Sequence[float], b: Sequence[float]) -> float:
    na = norm(a)
    nb = norm(b)
    if na == 0.0 or nb == 0.0:
        return 0.0
    return dot(a, b) / (na * nb)


def l2_energy(a: Sequence[float], b: Sequence[float]) -> float:
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(a, b)))


# ------------------------------
# Config, phases, trace, results
# ------------------------------

Predicate = Callable[['CompilerContext'], bool]


@dataclass
class OmegaPhase:
    name: str
    predicate: Predicate
    description: str = ""


@dataclass
class CompileConfig:
    max_depth: int = 256
    detect_cycles: bool = True
    max_states: int = 10000
    omega_phases: List[OmegaPhase] = field(default_factory=list)
    on_step: Optional[Callable[['TraceStep', 'RecursiveOntologyCompiler'], None]] = None


@dataclass
class TraceStep:
    depth: int
    symbol: str
    archetype: str
    ontic_layer: str
    similarity: Optional[float] = None
    energy: Optional[float] = None
    note: str = ""


@dataclass
class CompileResult:
    status: str  # "Ω-Complete" | "Ω-Overflow" | "Cycle-Detected" | "Dead-End" | "Guard-Limit"
    depth: int
    alpha: int
    omega_hits: int
    changed: bool
    phases_completed: List[str] = field(default_factory=list)
    trace: List[TraceStep] = field(default_factory=list)


@dataclass
class Snapshot:
    depth: int
    alpha: int
    omega_hits: int
    phases_completed: List[str]
    ontology_stack: List[str]


@dataclass
class CompilerContext:
    """Lightweight read-only context passed to predicates."""
    symbol: str
    archetype: str
    depth: int
    similarity: Optional[float]
    energy: Optional[float]
    compiler: 'RecursiveOntologyCompiler'


# ------------------------------
# Compiler
# ------------------------------

class RecursiveOntologyCompiler:
    def __init__(
        self,
        symbolic_alphabet: Dict[str, Any],
        ruleset: Dict[str, Any],
        config: Optional[CompileConfig] = None,
    ):
        self.symbols = symbolic_alphabet or {}
        self.rules = ruleset or {}
        self.cfg = config or CompileConfig()
        self.ontology_stack: List[str] = []
        self.alpha: int = 0
        self.omega_hits: int = 0
        self._visited: Set[Tuple[str, str]] = set()
        self._snapshots: List[Snapshot] = []
        self._phases_completed: List[str] = []

    # ---------- Hookable resolvers ----------

    def semantic_resolve(self, symbol: str) -> str:
        resolver = self.rules.get("semantic_resolve")
        if callable(resolver):
            return resolver(symbol, self.symbols, self.rules)
        lib = self.rules.get("archetypal_library", {})
        return lib.get(symbol, "Void-Function")

    def encode_ontology(self, archetype: str, depth: int) -> str:
        encoder = self.rules.get("encode_ontology")
        if callable(encoder):
            return encoder(archetype, depth, self.symbols, self.rules)
        return f"Onto[{archetype}] → Layer:{depth}"

    def transmute_symbol(self, archetype: str, depth: int) -> Optional[str]:
        transmuter = self.rules.get("transmute_symbol")
        if callable(transmuter):
            return transmuter(archetype, depth, self.symbols, self.rules)
        table = self.rules.get("transmutation_table", {})
        return table.get(archetype, "∞⊙∞")

    def vectorize(self, archetype: str) -> Optional[List[float]]:
        vec_fn = self.rules.get("vectorize")
        if callable(vec_fn):
            return vec_fn(archetype, self.symbols, self.rules)
        vecs = self.rules.get("vector_space", {})
        v = vecs.get(archetype)
        return list(v) if v is not None else None

    def omega_target_vector(self) -> Optional[List[float]]:
        target = self.rules.get("omega_target_vector")
        if callable(target):
            return target(self.symbols, self.rules)
        v = self.rules.get("omega_target_vector_value")
        return list(v) if v is not None else None

    # ---------- Snapshots and rollback ----------

    def snapshot(self, depth: int) -> Snapshot:
        snap = Snapshot(
            depth=depth,
            alpha=self.alpha,
            omega_hits=self.omega_hits,
            phases_completed=list(self._phases_completed),
            ontology_stack=list(self.ontology_stack),
        )
        self._snapshots.append(deepcopy(snap))
        return snap

    def rollback_last(self) -> Optional[Snapshot]:
        if not self._snapshots:
            return None
        snap = self._snapshots[-1]
        # Restore
        self.alpha = snap.alpha
        self.omega_hits = snap.omega_hits
        self._phases_completed = list(snap.phases_completed)
        self.ontology_stack = list(snap.ontology_stack)
        return deepcopy(snap)

    # ---------- Core compile loop ----------

    def compile(self, seed_symbol: str) -> CompileResult:
        # Reset state
        self.ontology_stack.clear()
        self.alpha = 0
        self.omega_hits = 0
        self._visited.clear()
        self._snapshots.clear()
        self._phases_completed.clear()

        symbol = seed_symbol
        trace: List[TraceStep] = []
        changed = False

        # Prepare metric target
        target_vec = self.omega_target_vector()

        # Phase pointer
        phase_idx = 0
        total_phases = len(self.cfg.omega_phases)

        for depth in range(self.cfg.max_depth + 1):
            archetype = self.semantic_resolve(symbol)
            ontic_layer = self.encode_ontology(archetype, depth)

            # Metric measurements (optional)
            similarity = None
            energy = None
            v = self.vectorize(archetype)
            if v is not None and target_vec is not None and len(v) == len(target_vec):
                similarity = cosine_similarity(v, target_vec)
                energy = l2_energy(v, target_vec)

            step = TraceStep(
                depth=depth,
                symbol=symbol,
                archetype=archetype,
                ontic_layer=ontic_layer,
                similarity=similarity,
                energy=energy,
            )
            self.ontology_stack.append(ontic_layer)
            trace.append(step)

            # Observability hook
            if callable(self.cfg.on_step):
                try:
                    self.cfg.on_step(step, self)
                except Exception:
                    # Non-fatal: keep compiling even if observer fails
                    pass

            # Cycle detection
            if self.cfg.detect_cycles:
                key = (symbol, archetype)
                if key in self._visited:
                    return CompileResult(
                        status="Cycle-Detected",
                        depth=depth,
                        alpha=self.alpha,
                        omega_hits=self.omega_hits,
                        changed=changed,
                        phases_completed=list(self._phases_completed),
                        trace=trace,
                    )
                self._visited.add(key)
                if len(self._visited) > self.cfg.max_states:
                    return CompileResult(
                        status="Guard-Limit",
                        depth=depth,
                        alpha=self.alpha,
                        omega_hits=self.omega_hits,
                        changed=changed,
                        phases_completed=list(self._phases_completed),
                        trace=trace,
                    )

            # Evaluate current and subsequent phases that may already be satisfied
            while phase_idx < total_phases:
                phase = self.cfg.omega_phases[phase_idx]
                ctx = CompilerContext(
                    symbol=symbol,
                    archetype=archetype,
                    depth=depth,
                    similarity=similarity,
                    energy=energy,
                    compiler=self,
                )
                if not phase.predicate(ctx):
                    break  # not yet satisfied
                # Phase satisfied: record and advance
                self.omega_hits += 1
                self.alpha += 1
                self._phases_completed.append(phase.name)
                if step.note:
                    step.note += f" | Ω: {phase.name}"
                else:
                    step.note = f"Ω: {phase.name}"
                # Save snapshot at each Ω gate
                self.snapshot(depth)
                phase_idx += 1
                # If there are more phases, loop to test the next immediately

            # All phases complete?
            if phase_idx >= total_phases and total_phases > 0:
                return CompileResult(
                    status="Ω-Complete",
                    depth=depth,
                    alpha=self.alpha,
                    omega_hits=self.omega_hits,
                    changed=True if (changed or total_phases > 0) else changed,
                    phases_completed=list(self._phases_completed),
                    trace=trace,
                )

            # Transmute to next symbol
            next_symbol = self.transmute_symbol(archetype, depth)
            if next_symbol != symbol:
                changed = True

            # Dead-end conditions
            if next_symbol in (None, "", "∞⊙∞"):
                return CompileResult(
                    status="Dead-End",
                    depth=depth,
                    alpha=self.alpha,
                    omega_hits=self.omega_hits,
                    changed=changed,
                    phases_completed=list(self._phases_completed),
                    trace=trace,
                )

            symbol = str(next_symbol)

        # If we exhausted depth without completing all phases
        return CompileResult(
            status="Ω-Overflow",
            depth=self.cfg.max_depth,
            alpha=self.alpha,
            omega_hits=self.omega_hits,
            changed=changed,
            phases_completed=list(self._phases_completed),
            trace=trace,
        )

    # ---------- Tracker rows and CSV ----------

    @staticmethod
    def rows(result: CompileResult) -> List[Dict[str, Any]]:
        return [
            {
                "depth": t.depth,
                "symbol": t.symbol,
                "archetype": t.archetype,
                "ontic_layer": t.ontic_layer,
                "similarity": "" if t.similarity is None else f"{t.similarity:.6f}",
                "energy": "" if t.energy is None else f"{t.energy:.6f}",
                "note": t.note,
            }
            for t in result.trace
        ]

    @staticmethod
    def write_csv(result: CompileResult, path: str) -> None:
        fieldnames = ["depth", "symbol", "archetype", "ontic_layer", "similarity", "energy", "note"]
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(RecursiveOntologyCompiler.rows(result))


# ------------------------------
# Demo rules and Ω phases
# ------------------------------

def default_rules() -> Dict[str, Any]:
    """
    Demo rules:
    - A → Creator → B
    - B → Destroyer → C
    - C → Self → C (fixed point)
    Vector space aligns archetypes to basis vectors; Ω target is Self.
    """
    ARCH_LIB = {"A": "Creator", "B": "Destroyer", "C": "Self"}

    TRANSMUTE_TABLE = {
        "Creator": "B",
        "Destroyer": "C",
        "Self": "C",  # remain at Self (stable)
    }

    VECTOR_SPACE = {
        "Creator": [1.0, 0.0, 0.0],
        "Destroyer": [0.0, 1.0, 0.0],
        "Self":     [0.0, 0.0, 1.0],
        "Void-Function": [0.0, 0.0, 0.0],
    }

    def encode_ontology(archetype: str, depth: int, *_):
        return f"[{depth:03d}] {archetype} ⇒ Structure"

    return {
        "archetypal_library": ARCH_LIB,
        "transmutation_table": TRANSMUTE_TABLE,
        "vector_space": VECTOR_SPACE,
        "omega_target_vector_value": VECTOR_SPACE["Self"],
        "encode_ontology": encode_ontology,
    }


def default_phases() -> List[OmegaPhase]:
    # Phase 1: semantic arrival at Self after at least 1 transition
    def phase1(ctx: CompilerContext) -> bool:
        return ctx.archetype == "Self" and ctx.depth >= 1

    # Phase 2: metric lock — near-perfect alignment with Self vector
    def phase2(ctx: CompilerContext) -> bool:
        if ctx.similarity is None or ctx.energy is None:
            return False
        return ctx.similarity >= 0.999 and ctx.energy <= 1e-6

    return [
        OmegaPhase(name="Semantic-Self", predicate=phase1, description="Reach archetype Self (post-seed)"),
        OmegaPhase(name="Vector-Lock", predicate=phase2, description="Cosine ~1.0 and near-zero L2 to Self vector"),
    ]


# ------------------------------
# Observability
# ------------------------------

def verbose_observer(step: TraceStep, _compiler: RecursiveOntologyCompiler) -> None:
    msg = f"Depth {step.depth:03d} | {step.symbol} → {step.archetype} | {step.ontic_layer}"
    if step.similarity is not None and step.energy is not None:
        msg += f" | sim={step.similarity:.6f} energy={step.energy:.6f}"
    if step.note:
        msg += f" | {step.note}"
    print(msg)


# ------------------------------
# Smoke tests
# ------------------------------

def smoke_tests() -> None:
    rules = default_rules()

    # 1) Convergence within a few steps
    cfg = CompileConfig(max_depth=16, omega_phases=default_phases())
    compiler = RecursiveOntologyCompiler({"A": "seed", "B": "", "C": ""}, rules, cfg)
    res = compiler.compile("A")
    assert res.status == "Ω-Complete", f"Expected Ω-Complete, got {res.status}"
    assert res.alpha == 2 and res.omega_hits == 2, f"Alpha/Omega mismatch: {res.alpha}/{res.omega_hits}"
    assert res.depth <= 3, f"Unexpectedly deep convergence: {res.depth}"
    assert "Semantic-Self" in res.phases_completed and "Vector-Lock" in res.phases_completed

    # 2) Cycle detection (oscillation that never reaches Self)
    rules_cycle = deepcopy(rules)
    rules_cycle["archetypal_library"] = {"X": "Creator", "Y": "Destroyer"}
    rules_cycle["transmutation_table"] = {"Creator": "Y", "Destroyer": "X"}
    cfg2 = CompileConfig(max_depth=32, detect_cycles=True, omega_phases=default_phases())
    compiler2 = RecursiveOntologyCompiler({"X": "", "Y": ""}, rules_cycle, cfg2)
    res2 = compiler2.compile("X")
    assert res2.status == "Cycle-Detected", f"Expected Cycle-Detected, got {res2.status}"

    # 3) Overflow (no Ω phases defined, long walk)
    cfg3 = CompileConfig(max_depth=5, omega_phases=[])
    compiler3 = RecursiveOntologyCompiler({"A": ""}, rules, cfg3)
    res3 = compiler3.compile("A")
    assert res3.status == "Ω-Overflow", f"Expected Ω-Overflow, got {res3.status}"

    # 4) Dead-end (transmutation returns None)
    rules_dead = deepcopy(rules)
    rules_dead["transmutation_table"] = {"Creator": None, "Destroyer": None, "Self": None}
    compiler4 = RecursiveOntologyCompiler({"A": ""}, rules_dead, CompileConfig(max_depth=5, omega_phases=default_phases()))
    res4 = compiler4.compile("A")
    assert res4.status == "Dead-End", f"Expected Dead-End, got {res4.status}"

    # 5) Rollback restores last Ω snapshot
    rules_rb = default_rules()
    cfg_rb = CompileConfig(max_depth=16, omega_phases=default_phases())
    compiler_rb = RecursiveOntologyCompiler({"A": ""}, rules_rb, cfg_rb)
    res_rb = compiler_rb.compile("A")
    assert res_rb.status == "Ω-Complete"
    pre_len = len(compiler_rb.ontology_stack)
    snap = compiler_rb.rollback_last()
    assert snap is not None
    post_len = len(compiler_rb.ontology_stack)
    assert post_len <= pre_len, "Rollback did not reduce or equalize ontology stack length"

    print("Smoke tests: OK")


# ------------------------------
# Main
# ------------------------------

def main(argv: Optional[List[str]] = None) -> int:
    parser = argparse.ArgumentParser(description="Recursive Ontology Compiler — demo run")
    parser.add_argument("--seed", default="A", help="Seed symbol to start compilation")
    parser.add_argument("--max_depth", type=int, default=16, help="Maximum steps")
    parser.add_argument("--csv", default="", help="Path to write CSV trace")
    parser.add_argument("--verbose", action="store_true", help="Print step-by-step trace")
    parser.add_argument("--no_tests", action="store_true", help="Skip smoke tests")
    args = parser.parse_args(argv)

    if not args.no_tests:
        smoke_tests()

    rules = default_rules()
    phases = default_phases()

    cfg = CompileConfig(
        max_depth=args.max_depth,
        omega_phases=phases,
        on_step=verbose_observer if args.verbose else None,
    )

    compiler = RecursiveOntologyCompiler(symbolic_alphabet={"A": "seed", "B": "", "C": ""}, ruleset=rules, config=cfg)
    result = compiler.compile(args.seed)

    print("\n— Run Summary —")
    print(f"Status: {result.status}")
    print(f"Depth: {result.depth}")
    print(f"Alpha: {result.alpha}")
    print(f"Omega hits: {result.omega_hits}")
    print(f"Phases completed: {', '.join(result.phases_completed) if result.phases_completed else '(none)'}")
    print(f"Changed: {result.changed}")

    if args.csv:
        RecursiveOntologyCompiler.write_csv(result, args.csv)
        print(f"Trace written to: {args.csv}")

    return 0


if __name__ == "__main__":
    sys.exit(main())